In [1]:
import sys
!{sys.executable} -m pip install numpy matplotlib pandas sympy scipy

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.show()
import matplotlib as mpl

pgf_with_rc_fonts = {
    "font.family": "serif",
}
mpl.rcParams.update(pgf_with_rc_fonts)

import pandas as pd
import sympy.functions.special
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection
import itertools

import pynoza
import scipy.interpolate
import scipy.optimize
import re
import importlib

%matplotlib notebook

In [22]:
filename = "../../../git_ignore/GLOBALEM/hira_at_obs_v1.txt"
data = pd.read_csv(filename,
                   delim_whitespace=True,
                   header=8)
names = ["Time", ]
with open(filename) as f:
    for i, line in enumerate(f):
        if i == 7:
            names.extend(line.split(" Electric field, ")[1:])
data.set_axis(names, axis=1, inplace=True)
data.head()

,Time,"x component (V/m), Point: (-2*r_HIRA, 0, 0)","y component (V/m), Point: (-2*r_HIRA, 0, 0)","z component (V/m), Point: (-2*r_HIRA, 0, 0)","x component (V/m), Point: (-2*r_HIRA, -2*r_HIRA, 0)","x component (V/m), Point: (0, -2*r_HIRA, 0)","x component (V/m), Point: (-2*r_HIRA, 2*r_HIRA, 0)","x component (V/m), Point: (0, 2*r_HIRA, 0)","x component (V/m), Point: (-2*r_HIRA, 0, -2*r_HIRA)","x component (V/m), Point: (0, 0, -2*r_HIRA)",...,"y component (V/m), Point: (0, 0, 2*r_HIRA)","z component (V/m), Point: (0, 0, 2*r_HIRA)","y component (V/m), Point: (-2*r_HIRA, -2*r_HIRA, 2*r_HIRA)","z component (V/m), Point: (-2*r_HIRA, -2*r_HIRA, 2*r_HIRA)","y component (V/m), Point: (0, -2*r_HIRA, 2*r_HIRA)","z component (V/m), Point: (0, -2*r_HIRA, 2*r_HIRA)","y component (V/m), Point: (-2*r_HIRA, 2*r_HIRA, 2*r_HIRA)","z component (V/m), Point: (-2*r_HIRA, 2*r_HIRA, 2*r_HIRA)","y component (V/m), Point: (0, 2*r_HIRA, 2*r_HIRA)","z component (V/m), Point: (0, 2*r_HIRA, 2*r_HIRA)\n"
0,1.000000e-11,8.884753e-28,-6.106765e-28,-8.784814e-28,3.085980e-27,4.796400e-21,3.722322e-34,3.692670e-31,-1.870748e-28,-9.737325e-25,...,0,0,9.970073e-28,2.343083e-29,0,0,5.207768e-33,4.923654e-33,-9.953633e-46,3.173888e-47
1,3.000000e-11,2.095018e-26,-1.100399e-26,-1.076588e-26,-2.839907e-25,-5.401215e-21,2.306121e-32,-3.242979e-30,-2.634395e-27,-2.662705e-24,...,0,0,-4.307558e-27,-1.315761e-27,0,0,-2.316630e-32,-3.884431e-32,-2.523741e-44,-2.472615e-45
2,6.333333e-11,5.247985e-27,5.277600e-27,-3.464697e-26,8.063177e-25,4.724209e-20,-1.157303e-31,4.276577e-30,1.037971e-25,1.527173e-23,...,0,0,6.891513e-27,2.270280e-27,0,0,1.950232e-31,2.159194e-31,6.474616e-44,2.746515e-44
3,9.666667e-11,5.476262e-25,5.838443e-25,-3.802255e-25,-5.139325e-25,1.257059e-19,1.061090e-31,1.813562e-29,1.083272e-24,8.072067e-22,...,0,0,1.189708e-27,-1.684294e-27,0,0,-3.408296e-32,-1.390760e-31,2.904446e-43,5.499296e-43
4,1.300000e-10,3.022484e-24,2.817010e-24,-1.657473e-24,-7.837622e-24,6.205480e-19,2.672390e-30,-1.946066e-29,2.829057e-24,2.501604e-22,...,0,0,-1.517167e-25,-4.342811e-26,0,0,-2.902888e-30,-4.733422e-30,2.547717e-43,3.165910e-42


In [94]:
r_HIRA = 0.7
shift = 0

src_range_x = [-r_HIRA, r_HIRA]
src_range_y = [shift - r_HIRA/2, shift - r_HIRA/2 + 2*r_HIRA]
src_range_z = [0, r_HIRA]

src_center = (np.mean(src_range_x), np.mean(src_range_y), np.mean(src_range_z))
src_radius = 2*r_HIRA

obs_range = [1.7*src_radius, 2*src_radius]


c0 = 3e8
t = np.array(data["Time"])
x1 = np.array([-2*r_HIRA, 0]).reshape((2, 1, 1))
x2 = np.array([-2*r_HIRA, 0, 2*r_HIRA]).reshape((1, 3, 1))
x3 = x2.reshape((1, 1, 3))
ex = np.zeros((x1.size, x2.size, x3.size, t.size))
ey, ez = ex.copy(), ex.copy()

def get_str_descr(x):
    if x > 0:
        return "2*r_HIRA"
    elif x < 0:
        return "-2*r_HIRA"
    else:
        return "0"

num_col = 0
for i, j, k in itertools.product(range(x1.size), range(x2.size), range(x3.size)):
    x1i, x2i, x3i = x1[i, 0, 0], x2[0, j, 0], x3[0, 0, k]
    str_descr = f"({get_str_descr(x1i)}, {get_str_descr(x2i)}, {get_str_descr(x3i)})"
    if x1i**2 + x2i**2 + x3i**2 < 1e-6: continue
    num_components = 0
    for i_col, col in enumerate(data.columns):
        if i_col == 0: continue
        if str_descr in col:
            if "x component" in col:
                ex[i, j, k, :] = data[col].values; num_components += 1
            elif "y component" in col:
                ey[i, j, k, :] = data[col].values; num_components += 1
            elif "z component" in col:
                ez[i, j, k, :] = data[col].values; num_components += 1
            else:
                raise RuntimeError(f"Unknown column: {col}")
    if num_components != 3:
        raise RuntimeError(f"Missing components for {str_descr} (has columns {data.columns})")
    num_col += 1
assert num_col == 17

array([[[[-2.08796899e-32, -8.28470804e-31,  8.99774622e-31, ...,
           1.39069089e-02,  1.40012115e-02,  1.35842242e-02],
         [ 4.12406161e-26,  3.15657796e-26, -2.37494155e-25, ...,
           4.88656078e-03,  2.79289136e-03,  6.49764723e-04],
         [ 2.34308306e-29, -1.31576139e-27,  2.27027999e-27, ...,
          -5.92763534e-03, -6.52148706e-03, -6.73077787e-03]],

        [[-1.39972033e-28, -4.13457104e-27,  1.66823220e-25, ...,
           1.56330340e-02,  1.36453330e-02,  1.10663904e-02],
         [-8.78481381e-28, -1.07658821e-26, -3.46469681e-26, ...,
          -3.87203725e-02, -3.52385904e-02, -2.71798065e-02],
         [ 1.55845151e-28,  4.58327441e-28, -5.97316467e-27, ...,
          -9.81571290e-03, -9.60429920e-03, -9.82290070e-03]],

        [[ 7.25643900e-36, -2.40435731e-35, -1.12102180e-33, ...,
          -2.14698666e-03, -3.30174660e-03, -4.14571914e-03],
         [ 5.62299552e-33, -1.96179956e-32,  3.13630076e-31, ...,
           3.52319956e-03,  5.9050

In [101]:
plt.close("all")
plt.figure()

plt.plot(t, ex[1, 0, 0, :])

<IPython.core.display.Javascript object>

In [4]:
M = 1
sol = pynoza.Solution(max_order=M,
                      wave_speed=1,)
sol.recurse()

def integrate_array(x):
    return np.cumsum(x)*dt

def derivative(x):
    return np.gradient(x, dt)

def get_all_orders(h):
    h_dict = {-1: integrate_array(h), 0: h}

    for i in range(1, M+3):
        h_dict[i] = derivative(h_dict[i-1])

    return h_dict

def get_fields(current_moment, charge_moment, h):

    h_dict = get_all_orders(h)

    c_mom = lambda a1, a2, a3: list(current_moment[a1, a2, a3])
    r_mom = lambda a1, a2, a3: list(charge_moment[a1, a2, a3])

    sol.set_moments(c_mom, r_mom)

    return sol.compute_e_field(x1,
                               x2,
                               x3,
                               t,
                               h_dict,
                               None)
e_true = [ex, ey, ez]

charge_moments = np.ones((sol.max_order+1, sol.max_order+1, sol.max_order+1, 3))
current_moments = charge_moments.copy()
Nmom = charge_moments.size
shape_mom = charge_moments.shape

h = np.sin(t)
Nh = h.size
shape_h = h.shape

def ravel_params(charge_moment, current_moments, h):
    return np.concatenate((np.ravel(charge_moment), np.ravel(current_moments), np.ravel(h)))

def unravel_params(params):
    return params[:Nmom].reshape(shape_mom), \
           params[Nmom:Nmom+Nmom].reshape(shape_mom), \
           params[2*Nmom:]

x0 = ravel_params(charge_moments, current_moments, h)

def get_error(x):

    current_moment, charge_moment, h = unravel_params(x)

    e_opt = get_fields(current_moment, charge_moment, h)

    error = 0
    for c1, c2 in zip(e_true, e_opt):
        error += np.sum((c1 - c2)**2)

    return error

get_error(x0)

#scipy.optimize.minimize(get_error, x0)

ValueError: non-broadcastable output operand with shape (501,) doesn't match the broadcast shape (50,50,50,501)

In [11]:
plt.close("all")
plt.figure()

h = np.sin(t)
hs = get_all_orders(h)

for order in hs:
    plt.plot(hs[order], label=f"h{order}")

plt.legend()

<IPython.core.display.Javascript object>

In [17]:
np.cumsum()

6.6666e-11